In [17]:
from aes import AES
from PIL import Image
import os
import random
from math import log, floor
import uuid

In [18]:
key = os.urandom(16)
print(f"key: {key}")
iv = os.urandom(16)
print(f"iv: {iv}")

key: b'\xa2H\xda1\xf0.\xa2\x802\x88\xc99/\xfe>|'
iv: b'\x1f\x15\xdfD\x84\xb3K\x85\xa9Z\x1aF\xbe\x85\x1d1'


In [19]:
def create_rgb_arrays(height, width, pixel_data):
    r_arr = []
    g_arr = []
    b_arr = []
    for y in range(height):
        for x in range(width):
            r,g,b = pixel_data[x,y]
            r_arr.append(r)
            g_arr.append(g)
            b_arr.append(b)
    return r_arr, g_arr, b_arr

def encrypt_array(arr, key, iv,mode: int):
    bytes_arr = bytearray(arr)
    encrypted_bytes:bytes = b""
    if mode == 1:
        encrypted_bytes = AES(key).encrypt_cbc(bytes_arr,iv)
    elif mode == 2:
        encrypted_bytes = AES(key).encrypt_pcbc(bytes_arr,iv)
    elif mode == 3:
        encrypted_bytes = AES(key).encrypt_cfb(bytes_arr,iv)
    elif mode == 4:
        encrypted_bytes = AES(key).encrypt_ofb(bytes_arr,iv)
    elif mode == 0:
        encrypted_bytes = AES(key).encrypt(bytes_arr)

    return encrypted_bytes

def create_rgb_encrypted_img(path:str,key:bytes,iv:bytes,mode:int):
    im = Image.open(path)
    pixel_data = im.load()

    width, height = im.size
    print(f"width: {width}, height: {height}")    
    r_arr, g_arr, b_arr = create_rgb_arrays(height, width, pixel_data)

    r_encrypted = encrypt_array(r_arr, key, iv,mode)
    g_encrypted = encrypt_array(g_arr, key, iv,mode)
    b_encrypted = encrypt_array(b_arr, key, iv,mode)

    print("encrypted bytes len: ", r_encrypted.__len__())
    print("plain bytes len: ",r_arr.__len__())
    print("needed width: ", width - (r_encrypted.__len__() - r_arr.__len__()))
    
    need_width = width - (r_encrypted.__len__() - r_arr.__len__())
    
    need_width_exp = floor(log(need_width,255))
    need_width_quo = need_width // pow(255,need_width_exp)
    need_width_rem = need_width % pow(255,need_width_exp)

    print("add_pixel: ", (need_width_exp,need_width_quo,need_width_rem))

    encrypted_img = Image.new("RGB", (width, height+1))
    for y in range(height+1):
        for x in range(width):
            if x+y*width < r_encrypted.__len__():
                encrypted_img.putpixel((x,y), (r_encrypted[x+y*width],g_encrypted[x+y*width],b_encrypted[x+y*width]))
            elif  x == width-1 and y == height:
                encrypted_img.putpixel((x,y), (need_width_exp,need_width_quo,need_width_rem))
            else:
                encrypted_img.putpixel((x,y), (random.randint(0,255),random.randint(0,255),random.randint(0,255)))
    return encrypted_img

In [20]:

path = str(input("Enter image path: "))
print("Select mode of operation:")
print("0. No mode of operation")
print("1. CBC")
print("2. PCBC")
print("3. CFB")
print("4. OFB")
mode = int(input("Enter mode: "))
encrypt_path = str(uuid.uuid4()) + ".png"
print(f"encrypt_path: {encrypt_path}")

encrypted_img = create_rgb_encrypted_img(path,key,iv,mode)
encrypted_img.save(encrypt_path)
print("encrypted image saved")


Select mode of operation:
0. No mode of operation
1. CBC
2. PCBC
3. CFB
4. OFB
encrypt_path: 414f1898-d9ba-42de-8dfa-c4e1f25dd3e8.png
width: 586, height: 485
encrypted bytes len:  284224
plain bytes len:  284210
needed width:  572
add_pixel:  (1, 2, 62)
encrypted image saved


In [21]:

def decrypt_array(arr, key, iv,mode: int):
    bytes_arr = bytearray(arr)
    decrypted_bytes:bytes = b""
    if mode == 1:
        decrypted_bytes = AES(key).decrypt_cbc(bytes_arr,iv)
    elif mode == 2:
        decrypted_bytes = AES(key).decrypt_pcbc(bytes_arr,iv)
    elif mode == 3:
        decrypted_bytes = AES(key).decrypt_cfb(bytes_arr,iv)
    elif mode == 4:
        decrypted_bytes = AES(key).decrypt_ofb(bytes_arr,iv)
    elif mode == 0:
        decrypted_bytes = AES(key).decrypt(bytes_arr)
        
    return decrypted_bytes

def decrypt_encrypted_img(path:str, key:bytes, iv:bytes,mode:int):
    encrypted_img = Image.open(path)
    width,height = encrypted_img.size
    print(f"width: {width} height: {height}")
    pixel_data = encrypted_img.load()
    need_width_exp,need_width_quo,need_width_rem = pixel_data[width-1,height-1]
    print(f"need_width_exp: {need_width_exp}, need_width_quo: {need_width_quo}, need_width_rem: {need_width_rem}")
    need_width = need_width_rem + need_width_quo*pow(255,need_width_exp)
    print("need_width: ", need_width)

    r_encrypted_bytes = []
    g_encrypted_bytes = []
    b_encrypted_bytes = []

    for y in range(height):
        for x in range(width):
            r,g,b = pixel_data[x,y]
            if x+y*width < width*height - need_width:
                r_encrypted_bytes.append(r)
                g_encrypted_bytes.append(g)
                b_encrypted_bytes.append(b)
            else:
                break
    
    r_decrypted_bytes = decrypt_array(r_encrypted_bytes, key, iv,mode)
    g_decrypted_bytes = decrypt_array(g_encrypted_bytes, key, iv,mode)
    b_decrypted_bytes = decrypt_array(b_encrypted_bytes, key, iv,mode)

    decrypt_width = width
    decrypt_height = height - 1 if need_width > 0 else height
    decrypted_img = Image.new("RGB", (decrypt_width, decrypt_height))

    for y in range(decrypt_height):
        for x in range(decrypt_width):
            decrypted_img.putpixel((x,y),(r_decrypted_bytes[x+y*width],g_decrypted_bytes[x+y*width],b_decrypted_bytes[x+y*width]))
            
    return decrypted_img


In [23]:

encrypt_path = str(input("Enter encrypted image path: "))
encrypt_path_name = ''.join(encrypt_path.split('.png'))

decrypt_path = f"{encrypt_path_name}_decrypted.png"
print(f"decrypt_path: {decrypt_path}")

print("Select mode of operation:")
print("0. No mode of operation")
print("1. CBC")
print("2. PCBC")
print("3. CFB")
print("4. OFB")
mode = int(input("Enter mode: "))

decrypted_img = decrypt_encrypted_img(encrypt_path,key,iv,mode)
decrypted_img.save(decrypt_path)
print("decrypted image saved")


decrypt_path: 414f1898-d9ba-42de-8dfa-c4e1f25dd3e8_decrypted.png
Select mode of operation:
0. No mode of operation
1. CBC
2. PCBC
3. CFB
4. OFB
width: 586 height: 486
need_width_exp: 1, need_width_quo: 2, need_width_rem: 62
need_width:  572
decrypted image saved
